In [ ]:
%pip install scikit-posthocs

In [ ]:
import pandas as pd
import scipy.stats as stats
import scikit_posthocs as sp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib  # 日本語表示用


In [ ]:
def visualize_distributions(df, target_col, group_col):
    """
    2群の分布形状を比較するための3つのグラフを描画する関数
    1. ヒストグラム + KDE（密度推定）: 直感的な形状把握
    2. ECDF（経験的累積分布関数）: 小標本でも情報の損失がない厳密な比較
    3. バイオリンプロット: 密度と範囲の比較
    """
    
    # データを抽出
    clean_df = df.dropna(subset=[target_col, group_col])
    groups = clean_df[group_col].unique()
    
    if len(groups) != 2:
        print("警告: この可視化は主に2群比較用です。")

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # --- 1. ヒストグラム + KDE (形状の確認) ---
    # KDE=Trueにすることで滑らかな近似曲線を描画し、形状比較をしやすくする
    sns.histplot(data=clean_df, x=target_col, hue=group_col, 
                 kde=True, element="step", stat="density", common_norm=False,
                 ax=axes[0], palette="Set2", alpha=0.3)
    axes[0].set_title(f"1. 分布形状 (Histogram + KDE)\n{target_col}")
    
    # --- 2. ECDF (累積分布関数) - 小標本に推奨 ---
    # Nが少ない時、ヒストグラムよりもこちらの方が「データの全容」を正確に表す
    sns.ecdfplot(data=clean_df, x=target_col, hue=group_col, 
                 linewidth=2, palette="Set2", ax=axes[1])
    axes[1].set_title(f"2. 累積分布 (ECDF)\n{target_col}")
    axes[1].set_ylabel("割合 (Cumulative)")
    axes[1].grid(True, linestyle='--', alpha=0.6)
    
    # --- 3. バイオリンプロット (密度の比較) ---
    # 箱ひげ図の情報に「幅（密度）」を加えたもの
    sns.violinplot(data=clean_df, x=group_col, y=target_col, hue=group_col,
                   palette="Set2", ax=axes[2], inner="quartile", alpha=0.6)
    sns.stripplot(data=clean_df, x=group_col, y=target_col, 
                  color='black', size=4, jitter=True, ax=axes[2])
    axes[2].set_title(f"3. 密度比較 (Violin Plot)\n{target_col}")
    
    plt.tight_layout()
    plt.show()

In [ ]:
def run_brunner_munzel(df, target_col, group_col):
    """
    Brunner-Munzel検定を実行する関数
    （等分散性を仮定しない、U検定の改良版）
    """
    print(f"{'='*60}")
    print(f" Brunner-Munzel検定: {target_col} by {group_col}")
    print(f"{'='*60}")
    
    # 1. データ準備（欠損除去）
    clean_df = df.dropna(subset=[target_col, group_col]).copy()
    groups = clean_df[group_col].unique()
    
    # 2群比較のみ対応
    if len(groups) != 2:
        print(f"エラー: グループ数が2ではありません（{len(groups)}群）。処理を中断します。")
        return

    g1_label, g2_label = groups[0], groups[1]
    x = clean_df[clean_df[group_col] == g1_label][target_col]
    y = clean_df[clean_df[group_col] == g2_label][target_col]
    
    n1, n2 = len(x), len(y)
    
    # 2. 検定実行
    # distribution='t': 小標本(N<50)の場合はt分布近似を使用するのが一般的
    w_stat, p_val = stats.brunnermunzel(x, y, alternative='two-sided', distribution='t')
    
    # 3. 確率推定値 (P_hat) の計算
    # P(X < Y) + 0.5 * P(X = Y) のこと。
    # 0.5なら「差なし」。0.5より大きければ群2が大きい、小さければ群1が大きい。
    # Brunner-Munzel検定統計量との関係から逆算も可能ですが、
    # 解釈用にU検定の理論を使って簡易計算して表示します（傾向把握用）
    u_stat, _ = stats.mannwhitneyu(x, y, alternative='two-sided')
    p_hat = u_stat / (n1 * n2)  
    # ※厳密にはBM検定独自の推定法がありますが、解釈はこれで十分通じます

    # 4. 結果表示
    print(f"  群1({g1_label}): N={n1}, Median={x.median():.2f}, Var={np.var(x, ddof=1):.2f}")
    print(f"  群2({g2_label}): N={n2}, Median={y.median():.2f}, Var={np.var(y, ddof=1):.2f}")
    print("-" * 40)
    print(f"  統計量(W): {w_stat:.4f}")
    print(f"  p値      : {p_val:.4f}")
    print(f"  確率推定(p̂): {p_hat:.3f} (0.5から離れるほど差が大きい)")
    
    if p_val < 0.05:
        print("  >> 判定: 有意差あり **")
        if p_hat < 0.5:
            print(f"     ({g2_label} の方が値が大きい傾向)")
        else:
            print(f"     ({g1_label} の方が値が大きい傾向)")
    else:
        print("  >> 判定: 有意差なし")
        
    return p_val

In [ ]:
def plot_stat_results(df, target_col, group_col, p_val=None, effect_size=None, title_suffix=""):
    """
    箱ひげ図 + ストリッププロットを描画し、検定結果をタイトルに表示する関数
    """
    plt.figure(figsize=(8, 6))
    
    # 1. 箱ひげ図 (分布の要約)
    # 色は薄く、外れ値(fliers)は非表示にする（ストリッププロットで全点打つため）
    sns.boxplot(data=df, x=group_col, y=target_col, 
                showfliers=False, color='white', linewidth=1.5, width=0.6)
    
    # 2. ストリッププロット (個別のデータ点)
    # jitter=Trueで点を少し散らして重なりを防ぐ
    sns.stripplot(data=df, x=group_col, y=target_col, 
                  color='#333333', size=7, jitter=0.15, alpha=0.7)
    
    # タイトルの生成（検定結果があれば表示）
    title_text = f"{group_col}別: {target_col} の比較"
    stats_text = ""
    
    if p_val is not None:
        significance = " (有意差あり **)" if p_val < 0.01 else " (有意差あり *)" if p_val < 0.05 else " (有意差なし)"
        stats_text += f"\np={p_val:.4f}{significance}"
    
    if effect_size is not None:
        # 効果量の表記 (r または η²)
        label = "r" if len(df[group_col].unique()) == 2 else "η²"
        stats_text += f", 効果量({label})={effect_size:.3f}"
        
    plt.title(title_text + stats_text + title_suffix, fontsize=14)
    plt.ylabel(target_col, fontsize=12)
    plt.xlabel(group_col, fontsize=12)
    
    # グリッド線（Y軸のみ）
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    
    plt.tight_layout()
    plt.show()

# --- 3群以上の事後検定用：ヒートマップ ---
def plot_dunn_heatmap(dunn_df, target_col):
    """Dunn検定のP値をヒートマップで表示"""
    plt.figure(figsize=(6, 5))
    sns.heatmap(dunn_df, annot=True, fmt=".4f", cmap="Blues_r", vmin=0, vmax=0.1)
    plt.title(f"事後検定(Dunn's Test) p値: {target_col}")
    plt.tight_layout()
    plt.show()

In [ ]:
def run_non_parametric_tests(df, target_col, group_col):
    """
    群の数に応じて、Mann-Whitney U検定（2群）または
    Kruskal-Wallis検定 + Dunn検定（3群以上）を自動で使い分ける関数
    """
    
    # 1. 前処理：データ抽出とクリーニング
    clean_df = df.dropna(subset=[target_col, group_col]).copy()
    groups = clean_df[group_col].unique()
    
    # グループごとのデータをリスト化
    data_list = [clean_df[clean_df[group_col] == g][target_col] for g in groups]
    
    print(f"{'='*60}")
    print(f" ノンパラメトリック検定: {target_col} by {group_col}")
    print(f" グループ数: {len(groups)} {groups}")
    print(f"{'='*60}")

    # --- ケース1: 2群の場合 (Mann-Whitney U検定) ---
    if len(groups) == 2:
        group1, group2 = groups[0], groups[1]
        x = clean_df[clean_df[group_col] == group1][target_col]
        y = clean_df[clean_df[group_col] == group2][target_col]
        
        # 検定実行 (alternative='two-sided' は両側検定)
        u_stat, p_val = stats.mannwhitneyu(x, y, alternative='two-sided')
        
        # 効果量 r の計算 (Z / sqrt(N))
        # scipyのmannwhitneyuはZ値を直接返さないため、正規近似を用いて算出
        n1, n2 = len(x), len(y)
        mu = n1 * n2 / 2
        sigma = np.sqrt(n1 * n2 * (n1 + n2 + 1) / 12)
        z = (u_stat - mu) / sigma
        r = abs(z) / np.sqrt(n1 + n2)

        print("\n【Mann-Whitney U検定 結果】")
        print(f"  群1({group1}): N={n1}, Median={x.median():.2f}")
        print(f"  群2({group2}): N={n2}, Median={y.median():.2f}")
        print("-" * 40)
        print(f"  U値: {u_stat:.2f}")
        print(f"  p値: {p_val:.4f}")
        print(f"  効果量(r): {r:.3f}")
        
        if p_val < 0.05:
            print("  >> 判定: 有意差あり (中央値に差がある)")
        else:
            print("  >> 判定: 有意差なし")
        
        plot_stat_results(clean_df, target_col, group_col, p_val=p_val, effect_size=r)

    # --- ケース2: 3群以上の場合 (Kruskal-Wallis検定) ---
    elif len(groups) >= 3:
        # 検定実行
        h_stat, p_val = stats.kruskal(*data_list)
        
        # 効果量 epsilon-squared (H / ((N^2 - 1) / (N + 1))) 
        # または単純に eta-squared = (H - k + 1) / (N - k)
        N = len(clean_df)
        eta_sq = (h_stat - len(groups) + 1) / (N - len(groups))

        print("\n【Kruskal-Wallis検定 結果】")
        for g in groups:
            med = clean_df[clean_df[group_col] == g][target_col].median()
            n = len(clean_df[clean_df[group_col] == g])
            print(f"  群({g}): N={n}, Median={med:.2f}")
        print("-" * 40)
        print(f"  H値: {h_stat:.2f}")
        print(f"  p値: {p_val:.4f}")
        print(f"  効果量(η²): {eta_sq:.3f}")

        plot_stat_results(clean_df, target_col, group_col, p_val=p_val, effect_size=eta_sq)

        if p_val < 0.05:
            print("  >> 判定: 有意差あり (少なくとも1つの群が異なる)")
            print("\n【事後検定: Dunn's Test (Bonferroni補正)】")
            plot_dunn_heatmap(dunn, target_col)
            
            # scikit-posthocsによるDunn検定
            dunn = sp.posthoc_dunn(clean_df, val_col=target_col, group_col=group_col, p_adjust='bonferroni')
            
            # 見やすいように整形して表示
            # p値が0.05未満のセルを探しやすいように表示します
            print(dunn)
            
            # 有意なペアをリストアップ
            print("\n  [有意差のあるペア]")
            signif_found = False
            cols = dunn.columns
            for i in range(len(cols)):
                for j in range(i+1, len(cols)):
                    p = dunn.iloc[i, j]
                    if p < 0.05:
                        print(f"  ・{cols[i]} vs {cols[j]} (p={p:.4f})")
                        signif_found = True
            if not signif_found:
                print("  ・(補正の結果、有意なペアは特定できませんでした)")

        else:
            print("  >> 判定: 有意差なし")
            
    else:
        print("エラー: グループ数が不足しています。")

In [ ]:
def plot_brunner_munzel(df, target_col, group_col):
    """
    Brunner-Munzel検定を行い、その結果（p値と有意差マーク）付きの
    箱ひげ図を描画する関数
    """
    # 1. データ準備
    clean_df = df.dropna(subset=[target_col, group_col]).copy()
    groups = clean_df[group_col].unique()
    
    if len(groups) != 2:
        print("エラー: 2群比較専用です。")
        return

    g1, g2 = groups[0], groups[1]
    d1 = clean_df[clean_df[group_col] == g1][target_col]
    d2 = clean_df[clean_df[group_col] == g2][target_col]
    
    # 2. Brunner-Munzel検定の実行
    # 小標本のためt分布近似を使用
    w_stat, p_val = stats.brunnermunzel(d1, d2, alternative='two-sided', distribution='t')
    
    # 3. グラフ描画設定
    plt.figure(figsize=(7, 6))
    
    # 箱ひげ図 (分布の要約) - 色を薄く
    sns.boxplot(data=clean_df, x=group_col, y=target_col, hue=group_col,
                showfliers=False, width=0.5, palette="Pastel1")
    
    # ストリッププロット (個別のデータ点) - 濃い色で
    # jitter=Trueで点を散らして重なりを防ぐ
    sns.stripplot(data=clean_df, x=group_col, y=target_col, hue=group_col,
                  palette="dark:#5A9_r", size=6, jitter=0.15, alpha=0.7)

    # 4. 有意差ライン（ブラケット）の描画
    # グラフの上限を取得して、その少し上に線を引く
    y_max = clean_df[target_col].max()
    y_min = clean_df[target_col].min()
    y_range = y_max - y_min
    
    # ラインの高さ設定
    h_line = y_max + (y_range * 0.05)  # データの最大値より5%上
    h_text = h_line + (y_range * 0.02) # 文字はそのさらに少し上
    
    # x軸の座標 (0, 1)
    x1, x2 = 0, 1
    
    # ブラケット（コの字型の線）を描く
    plt.plot([x1, x1, x2, x2], [h_line, h_line + (y_range*0.02), h_line + (y_range*0.02), h_line], lw=1.5, c='k')
    
    # 5. p値と判定マークの表示
    significance = "n.s." # 有意差なし (not significant)
    if p_val < 0.001: significance = "***"
    elif p_val < 0.01: significance = "**"
    elif p_val < 0.05: significance = "*"
    
    text_content = f"Brunner-Munzel test\np = {p_val:.4f}\n{significance}"
    
    plt.text((x1+x2)*0.5, h_text, text_content, ha='center', va='bottom', color='k', fontsize=12)

    # 6. タイトルと分散の表示（この検定を選んだ根拠として分散を表示）
    v1 = np.var(d1, ddof=1)
    v2 = np.var(d2, ddof=1)
    plt.title(f"{target_col} の群比較\n(Var: {g1}={v1:.1f}, {g2}={v2:.1f})", fontsize=14)
    
    # Y軸の範囲を調整（上の注釈が切れないように）
    plt.ylim(top=h_text + (y_range * 0.15))
    
    plt.ylabel(target_col)
    plt.xlabel(group_col)
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
df = pd.read_excel('AdvanceData.xlsx')

In [ ]:
visualize_distributions(df, 'env', 'role')

In [ ]:
run_non_parametric_tests(df, 'env', 'role')
run_brunner_munzel(df, 'env', 'role')
plot_brunner_munzel(df, 'env', 'role')